***Parse network and gene sets to proteinID***

In [121]:
import pandas as pd
import pickle

path = './files/'

Regulome Network from FunCoup

In [141]:
# import edge list ---- USE CYTOSCAPE TO VISUALIZE OR GRAPHISTRY
regulome_network = pd.read_csv("./files/humanRegulomeNetwork", delimiter="\t")
regulome_network_edges = regulome_network[['2:FunCoupAid', '3:FunCoupBid', '5:PPV', '18:LLR_GRG']]
regulome_network_edges.columns = ['ProteinAid', 'ProteinBid', 'PPV', 'LLR_GRG']
regulome_network_edges = regulome_network_edges.astype({'ProteinAid': 'str', 'ProteinBid':'str'})
regulome_network_edges.to_csv(path_or_buf='./files/human_regulome_pd.gz', compression='infer', index=False)

MSigDB - disease gene sets  

In [158]:
gsymbol_to_proteinid_dict = dict()
with open(path+'human_mapping', 'r') as map_file:
    for line in map_file.readlines():
        line = line.strip('\n').split('\t')
        if line[0] == 'Gene_Symbol':
            gsymbol_to_proteinid_dict[line[1]] = line[2]

def msigdb_json_to_dict(file): # very slow
    gene_set_dict = dict()
    with open(path+file, 'rb') as f:
        file_df = pd.read_json(f)
    gene_sets = file_df.loc['geneSymbols']
    for set in gene_sets.items():
        gene_set = pd.Series(data=set[1]).map(gsymbol_to_proteinid_dict).dropna()
        gene_set_dict[set[0]] = gene_set.to_list()
    with open(path+file.strip('.json')+'_dict.pkl', 'wb') as pkl_file:
        pickle.dump(gene_set_dict, pkl_file)


In [159]:
msigdb_json_to_dict('msigdb_c6_all.json')
msigdb_json_to_dict('msigdb_c2_cgp.json')
msigdb_json_to_dict('msigdb_c3_tft.json')